In [67]:
import pandas as pd
import numpy as np
import os
from datetime import timedelta

In [49]:
files = os.listdir('data/SP500Data/')

In [50]:
files_xls = [f for f in files if f[-4:] == 'xlsx']
# files_xls

In [82]:
to_split = df[df['ticker'] == 'A'][['Date','Adj Close']].set_index('Date')

In [135]:
def one_to_five(perc):
    if perc >= 0.2:
        return 5
    elif 0.2 > perc >= 0.03:
        return 4
    elif -0.03 < perc < 0.03:
        return 3
    elif -0.2 < perc <= -0.03:
        return 2
    else:
        return 1

In [139]:
def five_month_price_perc_diff(to_split,ticker):
    dct = {}
    perc_price_dif_lst = []
    month_lst = []
    month = to_split.index[0]
    for i in range(5):
        month_ini = month
        month = month + timedelta(30)
        if month in to_split.index:
            price2 = to_split.loc[month].iloc[0]
        elif (month + timedelta(1)) in to_split.index:
            month +=timedelta(1)
            price2 = to_split.loc[month+ timedelta(1)].iloc[0]
        elif (month + timedelta(2)) in to_split.index:
            month +=timedelta(2)
            price2 = to_split.loc[month+ timedelta(2)].iloc[0]
        else:
            month +=timedelta(3)
            price2 = to_split.loc[month+ timedelta(3)].iloc[0]

        price1 = to_split.loc[month_ini].iloc[0] 
        perc_price_dif = (price2-price1)/price1
        month_lst += [month]
        perc_price_dif_lst += [one_to_five(perc_price_dif)]
    dct['month'] = month_lst
    dct['perc_dif'] = perc_price_dif_lst
    toReturn = pd.DataFrame(dct)
    toReturn['ticker'] = ticker
    return toReturn

In [140]:
five_month_price_perc_diff(to_split,'A')

,month,perc_dif,ticker
0,2020-08-26,3,A
1,2020-09-25,3,A
2,2020-10-26,4,A
3,2020-11-25,4,A
4,2020-12-28,4,A


In [143]:
df = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel('data/SP500Data/'+ f)
    ticker = f[:-5]
    
    to_split = data[['Date','Adj Close']].set_index('Date')
    df = df.append(five_month_price_perc_diff(to_split,ticker))